In [133]:
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta, datetime
import talib as ta
from matplotlib.animation import FuncAnimation

In [134]:
class MovingAverageDayTrading():

    def __init__(self, ticker, stop_loss=0.05, take_profit=0.1):
        self.ticker = ticker
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        # This way, when an instance of the class is created, the self.stop_loss and self.take_profit will be set with default values of 0.05 and 0.1 respectively. These values can be overridden when an instance of the class is created by passing a different value for stop_loss and take_profit.

    def moving_average_timeframes(self):
        
        # set your API key
        quandl.ApiConfig.api_key = "hmd15bsczc3JdG4GLM4x"
        
        #  QUERYING DATA --------------------------------------------
        # end_date = datetime.now()
        # start_date = end_date - timedelta(days=365)
        # Get the data from Quandl
        df = quandl.get("BSE/BOM512415")#, returns="pandas", sort_order='desc', start_date=start_date, end_date=end_date)
        print(df.tail())
        
        pd.to_datetime(df.index)
        df.set_index("Date", inplace=True)
        # check the interval of your dataframe by using the df.resample function with the same interval of data you want to work with
        # df.resample('1min')

        # CLEANING DATA ------------------------------------------
        # df.index = pd.DatetimeIndex(df.index)
        
        print(f'$$$$$$$$$$$$$$$$$${df.shape}')

        

        df = df.dropna()
        # fill the empty spaces with what we want to fill it out
        df.fillna(method='ffill', inplace=True)

        # df = df.between_time('09:30', '16:00')

        # Filter the data to only include rows within trading hours
        # df = df[(df.index.hour >= 9) & (df.index.hour <= 16)]
        # Remove any rows for weekends (Saturday and Sunday)
        # df = df[df.index.weekday < 5]
        # Select only the columns of interest

        #  SETTING UP DATA: ADDING COLUMNS FOR MA  --------------------------------------


        print(df)
        # Exponential Moving Averages
        df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
        print(df['EMA_20'])
        df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
        df['EMA_200'] = df['Close'].ewm(span=200, adjust=False).mean()
        # the ewm() method to calculate the EMA, passing in the window size or span, which is the number of periods to use in the moving average. You can also adjust whether or not to apply some decay to the weights by adjusting the adjust parameter. By default is set to True, but it depends on the use case.
        
        # Simple Moving Averages
        df['SMA_20'] = df['Close'].rolling(window=20).mean()
        df['SMA_50'] = df['Close'].rolling(window=50).mean()
        df['SMA_200'] = df['Close'].rolling(window=200).mean()
        
        # Bollinger Bands
        # df['BOL_upper'], df['BOL_middle'], df['BOL_lower'] = ta.BBANDS(df["Close"])
        
        # # Stochastic Oscillator
        # df['STO_slowk'], df['STO_slowd'] = ta.STOCH(df["High"], df["Low"], df["Close"])
        
        # # Relative Strength Index
        # df['RSI'] = ta.RSI(df["Close"])
        
        # # Moving Average Convergence Divergence
        # df['MACD'], df['MACD_signal'], df['MACD_hist'] = ta.MACD(df["Close"])

        df = df[['Close', 'EMA_20', 'EMA_50', 'EMA_200','SMA_20', 'SMA_50', 'SMA_200']]
        # The idea here is to now have access to various indicators and technical analysis for the current ticker and by having different timeframes for each one of the indicators (20,50,200) you can have a better understanding of the current state of the market.
        # Additionally you could use the `df` dataframe to plot these indicators, and analyze the data to identify potential buy and sell signals, and also use it to perform some backtesting to evaluate the performance of your strategy over time, as well as create stop loss and take profit conditions.
        # dates = df.values[:, 0]
        # print(dates)
        
        print(f'@@@@@@@@@@{df}@@@@@@@@@@')
        # print(df['datetime'])
        # Convert the 'Date' column to a datetime type
        # df['Date'] = pd.to_datetime(df['Date'])
        # Set the 'Date' column as the index of the DataFrame
        # df = df.set_index('datetime')
        # Remove any rows with missing values


        #  RISK MANAGEMENT // CHECKS BEFORE PLOTTING ----------------------------

        if self.stop_loss is None or self.take_profit is None:
            raise ValueError("stop_loss and take_profit values must be set before using them")
        
        if len(range(len(df))) == 0:
            raise ValueError("Frames data is empty.")
            
        if len(df) == 0:
            raise ValueError("Dataframe is empty.")


        # LOGIC TO SET UP BUY AND SELL SIGNALS
        buy = []
        sell = []
        # for i in range(1, len(df)):
        #     # Check for buy signal
        #     if (df['EMA_20'][i] > df['EMA_50'][i] and df['EMA_20'][i-1] < df['EMA_50'][i-1]) and (df['EMA_20'][i] > df['EMA_200'][i] and df['EMA_20'][i-1] < df['EMA_200'][i-1]) and (df['SMA_20'][i] > df['SMA_50'][i] and df['SMA_20'][i-1] < df['SMA_50'][i-1]) and (df['SMA_20'][i] > df['SMA_200'][i] and df['SMA_20'][i-1] < df['SMA_200'][i-1]):
        #         buy.append((df.index[i], df['Close'][i]))
        #         current_price = df['Close'][i]
        #         stop_loss_price = current_price * (1 - self.stop_loss)
        #         take_profit_price = current_price * (1 + self.take_profit)
                
        #         # Implement stop loss and take profit
        #         for j in range(i, len(df)):
        #             if df['Close'][j] < stop_loss_price:
        #                 sell.append((df.index[j], df['Close'][j]))
        #                 break
        #             elif df['Close'][j] > take_profit_price:
        #                 sell.append((df.index[j], df['Close'][j]))
        #                 break
        #     # Check for sell signal   
        #     elif (df['EMA_20'][i] < df['EMA_50'][i] and df['EMA_20'][i-1] > df['EMA_50'][i-1]) and (df['EMA_20'][i] < df['EMA_200'][i] and df['EMA_20'][i-1] > df['EMA_200'][i-1]) and (df['SMA_20'][i] < df['SMA_50'][i] and df['SMA_20'][i-1] > df['SMA_50'][i-1]) and (df['SMA_20'][i] < df['SMA_200'][i] and df['SMA_20'][i-1] > df['SMA_200'][i-1]):
        #         sell.append((df.index[i], df['Close'][i]))

        # PLOTTING WITH MATPLOTLIB 

        # plt.plot()
        # plt.figure(figsize=(12,5))
        # plt.xlabel("date")
        # plt.ylabel("price")
        # plt.plot(df['Close'], label= 'Asset price', c='blue', alpha=0.9)
        # plt.plot(df['EMA_20'], label='EMA_20', c='k', alpha= 0.9)
        # plt.plot(df['EMA_50'], label='EMA_50', c='magenta', alpha= 0.9)
        # plt.plot(df['EMA_200'], label='EMA_200', c='yellow', alpha= 0.9)
        # plt.plot(df['SMA_20'], label='SMA_20', c='#007900', alpha= 0.9)
        # plt.plot(df['SMA_50'], label='SMA_50', c='#00ff00', alpha= 0.9)
        # plt.plot(df['SMA_200'], label='SMA_200', c='#00aeae', alpha= 0.9)
        # plt.plot(df['BOL_upper'], label='BOL_upper', c='#ff7f00', alpha= 0.9)
        # plt.plot(df['STO_slowk'], label='STO_slowk', c='#7308a5', alpha= 0.9)
        # plt.plot(df['RSI'], label='RSI', c='#ffff00', alpha= 0.9)
        # plt.plot(df['MACD'], label='MACD', c='#feb300', alpha= 0.9)
        
        # plt.scatter(df.iloc[buy].index,df.iloc[buy]['Close'], marker='^', color='g', s=100)
        # plt.scatter(df.iloc[sell].index,df.iloc[sell]['Close'], marker='v', color='r', s=100)

        # plt.title('Day Trading')
        # plt.legend(shadow=True)
        # return plt.show()
        

        # Create a figure and a set of subplots
        fig, ax = plt.subplots(figsize=(12,5))
        plt.xlabel("date")
        plt.ylabel("price")

        # Initialize the lines that will be updated
        line_1, = ax.plot(df['Close'], label= 'Asset price', c='blue', alpha=0.9)
        line_2, = ax.plot(df['EMA_20'], label='EMA_20', c='k', alpha= 0.9)
        line_3, = ax.plot(df['EMA_50'], label='EMA_50', c='magenta', alpha= 0.9)
        line_4, = ax.plot(df['EMA_200'], label='EMA_200', c='yellow', alpha= 0.9)
        line_5, = ax.plot(df['SMA_20'], label='SMA_20', c='#007900', alpha= 0.9)
        line_6, = ax.plot(df['SMA_50'], label='SMA_50', c='#00ff00', alpha= 0.9)
        line_7, = ax.plot(df['SMA_200'], label='SMA_200', c='#00aeae', alpha= 0.9)
        # line_8, = ax.plot(df['BOL_upper'], label='BOL_upper', c='#ff7f00', alpha= 0.9)
        # line_9, = ax.plot(df['STO_slowk'], label='STO_slowk', c='#7308a5', alpha= 0.9)
        # line_10, = ax.plot(df['RSI'], label='RSI', c='#ffff00', alpha= 0.9)
        # line_11, = ax.plot(df['MACD'], label='MACD', c='#feb300', alpha= 0.9)
        scatter_1 = ax.scatter(df.iloc[buy].index,df.iloc[buy]['Close'], marker='^', color='g', s=100)
        scatter_2 = ax.scatter(df.iloc[sell].index,df.iloc[sell]['Close'], marker='v', color='r', s=100)
        plt.title('Day Trading')
        plt.legend(shadow=True)

        # Define the update function
        def update(frame):
            line_1.set_ydata(df['Close'][frame:])
            line_2.set_ydata(df['EMA_20'][frame:])
            line_3.set_ydata(df['EMA_50'][frame:])
            line_4.set_ydata(df['EMA_200'][frame:])
            line_5.set_ydata(df['SMA_20'][frame:])
            line_6.set_ydata(df['SMA_50'][frame:])
            line_7.set_ydata(df['SMA_200'][frame:])
            # line_8.set_ydata(df['BOL_upper'][frame:])
            # line_9.set_ydata(df['STO_slowk'][frame:])
            # line_10.set_ydata(df['RSI'][frame:])
            # line_11.set_ydata(df['MACD'][frame:])
            scatter_1.set_offsets(np.c_[df.iloc[buy].index[frame:],df.iloc[buy]['Close'][frame:]])
            scatter_2.set_offsets(np.c_[df.iloc[sell].index[frame:],df.iloc[sell]['Close'][frame:]])
            return line_1, line_2, line_3, line_4, line_5, line_6, line_7, line_8, line_9, line_10, line_11, scatter_1, scatter_2

        # create animation
        ani = FuncAnimation(fig, update, frames=range(len(df)), blit=True)

        # save animation
        ani.save("day_trading.gif", writer="imagemagick")
        return display(HTML("<img src='day_trading.gif'>"))


if __name__ == '__main__':
    
    average = MovingAverageDayTrading('INTC', stop_loss=0.03, take_profit=0.15)
    print(average.moving_average_timeframes())

            Open  High  Low  Close  WAP  No. of Shares  No. of Trades  \
Date                                                                    
1999-12-07   3.0   3.0  3.0    3.0  3.0         1000.0            1.0   
1999-12-08   3.0   3.0  3.0    3.0  3.0         1000.0            1.0   
1999-12-16   2.9   2.9  2.9    2.9  2.9         1500.0            1.0   
2006-11-24   3.0   3.0  3.0    3.0  3.0          100.0            1.0   
2021-02-25   3.1   3.1  3.1    3.1  3.1          200.0            2.0   

            Total Turnover  Deliverable Quantity  % Deli. Qty to Traded Qty  \
Date                                                                          
1999-12-07          3000.0                   NaN                        NaN   
1999-12-08          3000.0                   NaN                        NaN   
1999-12-16          4350.0                   NaN                        NaN   
2006-11-24           300.0                 100.0                      100.0   
2021-02-25    

KeyError: "None of ['Date'] are in the columns"